In [2]:
import numpy as np
import pandas as pd
import os 
import sys
sys.path.append(os.path.abspath('../common')) # add path to common functions

In [14]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)

In [15]:
train_df['input_str'] = train_df['input_str'].apply(lambda x : x[3:])
valid_df['input_str'] = valid_df['input_str'].apply(lambda x : x[3:])

In [16]:
num_feat = 47

def get_columns (df) :
    for i in range(num_feat):
        df[f'c_{i}'] = df['input_str'].apply(lambda x : x[i])
    return df.drop(columns = ['input_str'])

train_df = get_columns(train_df)
valid_df = get_columns(valid_df)

In [20]:
import pandas as pd

df = train_df

# Convert string values in the 47 columns to integers
df.iloc[:, :-1] = df.iloc[:, :-1].astype(int)  # Assuming last column is the target

# Initialize a DataFrame to store counts of each number (0-9)
count_df = pd.DataFrame(0, index=df.index, columns=[f'count_{i}' for i in range(10)])

# For each number (0-9), count its occurrences across the 47 columns
for i in range(10):
    count_df[f'count_{i}'] = (df.iloc[:, :-1] == i).sum(axis=1)

# Add the binary target column to the count_df for correlation calculation
count_df['label'] = df['label']  # Assuming 'target' is the name of the binary target column

# Calculate the Pearson correlation between the count of each number (0-9) and the target
correlations = count_df.corr()['label'].drop('label')

# Display the correlations
print(correlations)


count_0   -0.434806
count_1    0.378802
count_2    0.006309
count_3    0.002525
count_4   -0.016974
count_5   -0.020814
count_6   -0.000944
count_7    0.012974
count_8    0.027284
count_9   -0.000509
Name: label, dtype: float64


In [18]:
df

,label,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,...,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46
0,0,0,1,5,4,3,6,4,6,4,...,1,5,9,6,2,6,2,6,1,4
1,0,4,6,4,1,5,9,6,3,6,...,4,7,6,1,6,1,4,2,8,4
2,0,1,5,4,3,6,2,6,2,1,...,0,5,1,1,5,9,6,2,8,4
3,1,0,1,5,4,3,6,4,2,2,...,3,5,1,0,6,1,4,2,6,2
4,1,4,6,4,1,8,9,9,4,2,...,1,5,9,6,6,1,4,2,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,1,0,0,2,6,2,6,7,0,2,...,1,4,1,5,9,6,2,4,1,4
7076,1,0,4,6,4,4,2,2,1,5,...,6,1,4,3,9,0,1,2,6,2
7077,1,3,0,6,9,9,1,5,9,6,...,2,6,2,6,1,4,1,5,9,6
7078,0,0,6,1,4,1,5,9,6,1,...,6,9,4,0,6,1,4,2,8,4


In [29]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)
# train_df['input_str'] = train_df['input_str'].apply(lambda x : x[3:])
# valid_df['input_str'] = valid_df['input_str'].apply(lambda x : x[3:])

In [43]:
for index,row in train_df.iterrows() :
    # print(row['label'])
    # a = row['input_str']
    # i = 0
    # while a[i] == '0' :
    #     i += 1
    # print(a[i:])  
    # for i in range (8) :
    #     print(row[f'input_str'][4*i:4*(i+1)], end = ' ')
    # for i in range (8, 13) :
    #     print(row[f'input_str'][32 + 3*(i-8) : 32 + 3*(i-7)], end = ' ')
    # print()

SyntaxError: incomplete input (4098495728.py, line 12)

In [ ]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)

In [38]:
import tensorflow as tf
from tensorflow.keras import layers

# Number of unique digits (0-9) in the input and emojis (13 unique emojis) in the output
num_input_tokens = 10  # digits 0-9
num_output_emojis = 13  # the number of unique emojis
embedding_dim = 32  # Embedding size for each digit
hidden_units = 16  # LSTM/GRU units

# Model architecture
def build_seq2seq_model(input_length=50, output_length=13):
    # Input Layer (50-length sequence of digits 0-9)
    encoder_inputs = layers.Input(shape=(input_length,))
    
    # Embedding layer (Converting digit inputs into dense vectors)
    x = layers.Embedding(input_dim=num_input_tokens, output_dim=embedding_dim, input_length=input_length)(encoder_inputs)
    
    # Encoder (LSTM or GRU to encode the sequence of digits)
    encoder_outputs, state_h, state_c = layers.LSTM(hidden_units, return_state=True)(x)
    
    # Repeat the encoder's context vector (state) for the number of output emojis (13)
    decoder_inputs = layers.RepeatVector(output_length)(encoder_outputs)
    
    # Decoder (Using an LSTM or GRU to generate the sequence of emojis)
    decoder_lstm = layers.LSTM(hidden_units, return_sequences=True)(decoder_inputs, initial_state=[state_h, state_c])
    
    # Output Layer (Predicting emojis using a softmax layer)
    decoder_outputs = layers.TimeDistributed(layers.Dense(num_output_emojis, activation='softmax'))(decoder_lstm)
    
    # Build the model
    model = tf.keras.Model(encoder_inputs, decoder_outputs)
    
    return model

# Build the model
seq2seq_model = build_seq2seq_model()

# Compile the model (assuming we're using categorical crossentropy for multi-class classification)
seq2seq_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
seq2seq_model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 50, 32)    │        320 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_12 (LSTM)      │ [(None, 16),      │      3,136 │ embedding_6[0][0] │
│                     │ (None, 16),       │            │                   │
│                     │ (None, 16)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_6     │ (None, 13, 16)    │          0 │ lstm_12[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_13 (LSTM)      │ (None, 13, 16)    │      2,112 │ repeat_vector_6[… │
│                     │                   │            │ lstm_12[0][1],    │
│                     │                   │            │ lstm_12[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 13, 13)    │        221 │ lstm_13[0][0]     │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,789 (22.61 KB)

 Trainable params: 5,789 (22.61 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
x_train = train_df.drop(columns = ['label'])
x_valid = valid_df.drop(columns = ['label'])

In [46]:
y_train, y_valid = pd.read_csv('../datasets/train/train_emoticon.csv'), pd.read_csv('../datasets/valid/valid_emoticon.csv')

In [48]:
y_train = y_train.drop(columns = ['label'])
y_valid = y_valid.drop(columns = ['label'])

In [51]:
seq2seq_model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10, batch_size=32)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 1), output.shape=(None, 13, 13)